<h1>Prediction of beer consumption in the Czech Republic: ARIMA vs Prophet</h1>
<h3>Viktoriia Ilina</h3>
<h3>October 2021</h3>

<h2>Overview</h2>

The Czech Republic consumes more beer per capita than any other country in [the world](https://worldpopulationreview.com/country-rankings/beer-consumption-by-country/). Beating out Germany, Austria and Belgium, the country drinks over 140 liters of beer per person each year. However, 2020 was a dramatic year for the beer industry: consumption fell back to the level of 1960s (135 liters per capita). According to [the Czech Association of Breweries and Malthouses](https://www.expats.cz/czech-news/article/czech-beer-consumption-in-2020-fell-to-its-lowest-level-since-the-1960s/), coronavirus restrictions such as reduced opening hours of pubs and restaurants, a ban on alcohol consumption in public, and the cancellation of sports and cultural events  are to blame for the drop. Nevertheless, it was not enough to kick the country from the top spot. Undoubtedly, such record is a very controversial in terms of public health.

The main goal of this project is trying to predict annual beer consumption using classical statistical model (ARIMA) and Facebook Prophet. The dataset was extracted from the official website of the czech statistical office (CZSO) and contains information about annually beer intake per capita in the Czech Republic since 1948 and until 2019. 

<h2>Analysis</h2>

First, we install all necessary libraries and load the data.

In [2]:
!pip install pandas
import pandas as pd
!pip install plotly
!pip install cufflinks
import cufflinks as cf
from IPython.display import display,HTML
cf.set_config_file(sharing = 'public', theme = 'white', offline = True)
!pip install statsmodels
from statsmodels.tsa.stattools import adfuller, kpss
!pip install pmdarima
from pmdarima.arima import auto_arima
import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX
!pip install pystan==2.19.1.1 
!pip install fbprophet
from fbprophet import Prophet
!pip install sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tools.eval_measures import rmse
import warnings
warnings.filterwarnings('ignore')


filepath = 'https://github.com/Viktoriia-Ilina/Prediction-of-beer-consumption-in-the-Czech-Republic-ARIMA-vs-Prophet/blob/main/beerconsumption.csv?raw=true'
data = pd.read_csv(filepath)
data.head()

     |████████████████████████████████| 81 kB 13.0 MB/s eta 0:00:01


  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=67921 sha256=641f9c5aae62630736f1cb58986b5f494a418b349ac7781d339967b132814f6d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/6b/76/62/6da97734911ffcbdd559fd1a3f28526321f0ae699182a23866
Successfully built cufflinks


     |████████████████████████████████| 1.5 MB 18.9 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 55.3 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.18.0 requires nose, which is not installed.
hdijupyterutils 0.18.0 requires jupyter>=1, which is not installed.
hdijupyterutils 0.18.0 requires nose, which is not installed.
tensorflow 2.4.3 requires flatbuffers~=1.12.0, but you have flatbuffers 20210226132247 which is incompatible.
tensorflow 2.4.3 requires grpcio~=1.32.0, but you have grpcio 1.35.0 which is incompatible.
tensorflow 2.4.3 requires numpy~=1.19.2, but you have numpy 1.21.3 which is incompatible.
tensorflow 2.4.3 requires opt-einsum~=3.3.0, but you hav

  ERROR: Command errored out with exit status 1:
   command: /opt/conda/envs/Python-3.8-main/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/wsuser/pip-install-5v57n5dx/fbprophet_57839ea42e8741ebb837647981d590b3/setup.py'"'"'; __file__='"'"'/tmp/wsuser/pip-install-5v57n5dx/fbprophet_57839ea42e8741ebb837647981d590b3/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/wsuser/pip-wheel-si58cvf9
       cwd: /tmp/wsuser/pip-install-5v57n5dx/fbprophet_57839ea42e8741ebb837647981d590b3/
  Complete output (44 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/fbprophet
  creating build/lib/fbprophet/stan_model
  Traceback (most recent call last):
    

,Date,Consumption
0,1948-12-31,76.2
1,1949-12-31,90.2
2,1950-12-31,99.4
3,1951-12-31,101.4
4,1952-12-31,101.0


Let's convert 'Date' column into the right formart and then set it as index of data frame.

In [3]:
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')
data

,Consumption
Date,
1948-12-31,76.2
1949-12-31,90.2
1950-12-31,99.4
1951-12-31,101.4
1952-12-31,101.0
...,...
2015-12-31,146.6
2016-12-31,146.9
2017-12-31,144.3


Here, we have an information about annually beer consumption per capita in the Czech Republic since 1948 and until 2019. Incredible 163.5 litres per person were drank in 2005, the most sober year was 1948. Consumption  was rapidly going up through 1954-1971, then stagnated for several decades, and tending to go down the last ten years. 

In [4]:
data.iplot(dimensions = (800,500), xTitle = 'Year', yTitle = 'Litres per capita', title = 'Beer consumption in Czech Republic 1948-2019', color = '#008000')

And now let us move on to modelling and forecasting.

<h3>ARIMA</h3>

[ARIMA](https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/), short for ‘AutoRegressive Integrated Moving Average’, is a forecasting algorithm based on the idea that the information in the past values of the time series can alone be used to predict the future values.

[The parameters](https://otexts.com/fpp2/non-seasonal-arima.html/) of the ARIMA model are defined as follows: 

- p: the order of the autoregressive part;
- d: the number of differencing required to make the time series stationary;
- q: the order of the moving average part.

The crutial step in building an ARIMA model is check stationarity of initial series.  [Stationarity](https://dspace.cuni.cz/bitstream/handle/20.500.11956/124617/120381341.pdf/) stands for statistical properties of a time series – autocorrelation, expectation, variance – staying
unchanged over time. In reality however, time series often contain trends, random walks, periodic fluctuations, etc., or even a combination of those, which essentially makes them non-stationary by mature. 
Nonetheless, non-stationary data, as a rule, are unpredictable and cannot be modeled or forecasted. Thus, for making effective and precise predictions, non-stationary series must be transformed into stationary using different data transformation methods.

We runs formal stationarity testing using the Augmented Dickey-Fuller (ADF) and the Kwiatkowski–Phillips–Schmidt–Shin (KPSS) tests. The null hypothesis of ADF implies that a unit root against the alternative of no unit root. The null hypothesis of KPPS, in its turn, consists in stationarity against an alternative of a unit root. Using two tests in a tandem allows to reveal, whether the explored series is indeed stationary.

In [ ]:
adf = adfuller(data['Consumption'], autolag='AIC')
print(f'ADF Statistic: {adf[0]}')
print(f'p-value: {adf[1]}')

In [6]:
kpss = kpss(data['Consumption'], regression='c')
print('KPSS Statistic: %f' % kpss[0])
print('p-value: %f' % kpss[1])

KPSS Statistic: 0.445865
p-value: 0.057386


We observe the difference in the results from ADF test and KPSS test (ADF indicates stationarity, whereas KPSS indicates non-stationarity), so our data is not strict stationary. [Differencing](https://dspace.cuni.cz/bitstream/handle/20.500.11956/124617/120381341.pdf/) may be useful in terms of stabilizing the mean of time series through eliminating the changes in time series level, and so excluding – or at least mitigating – trend. Hence, we should implement ARIMA model where 'd' must be greater than or equal to one. To get best parameters for the model, we use auto_arima function() for entire dataset. Function select the model with the least number of parameters that give the lowest AIC compared to other [models](https://towardsdatascience.com/advanced-time-series-analysis-with-arma-and-arima-a7d9b589ed6d).

In [7]:
model = pm.auto_arima(data['Consumption'], 
                      start_p = 0, start_q = 0,
                      test = 'kpss',       
                      max_p = 5, max_q = 5, 
                      d = None,          
                      seasonal = False,   
                      trace = True,
                      error_action = 'ignore',  
                      suppress_warnings = True, 
                      stepwise = True)

print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=494.460, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=465.455, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.06 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=492.502, Time=0.01 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=465.652, Time=0.04 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.11 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.14 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=463.579, Time=0.01 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=463.803, Time=0.02 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=451.829, Time=0.05 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=450.000, Time=0.02 sec
 ARIMA(0,2,2)(0,0,0)[0]             : AIC=451.870, Time=0.03 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=inf, Time=0.12 sec

Best model:  ARIMA(0,2,1)(0,0,0)[0]          
Total fit time: 0.643 seconds
                               SARIMAX Results     

According to results of Ljung-Box test, the residuals are independently distributed. Nevertheless, Jarque-Bera test failed, that means a non-normal distribution of residuals. Sometimes applying a Box-Cox transformation may assist with this [problem](https://otexts.com/fpp2/residuals.html), but overall there are no guarantees. In practice, it's a common situation when the model doesn't pass all of the residual tests. We could use this model, but we should keep in mind it may affect the coverage probability of the prediction intervals. 

Let’s split the data into train and test set (68 and 5 years correspondingly). Using the parameters from the previous step, we are trying to forecast the consumption.

In [8]:
train = data[:len(data)-5]
test = data[len(data)-5:]

arima_model = SARIMAX(train['Consumption'], order = (0,2,1))
arima_result = arima_model.fit()
arima_pred = arima_result.forecast(steps = 5, index = test.index).rename('ARIMA Predictions')
test['ARIMA_Predictions'] = arima_pred.values
test

,Consumption,ARIMA_Predictions
Date,,
2015-12-31,146.6,146.705131
2016-12-31,146.9,146.410262
2017-12-31,144.3,146.115393
2018-12-31,145.2,145.820524
2019-12-31,146.0,145.525655


It looks too good to be true: predicted values are very closed to real test values. 

<h3>Prophet</h3>

[Prophet](https://facebook.github.io/prophet/) is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

The input to Prophet is always a dataframe with two columns: ds and y. Therefore, we should prepare our initial dataset and then split it into train and test set. 

In [12]:
data_pr = data.copy()
data_pr = data.reset_index()
data_pr.columns = ['ds','y'] 
train_pr = data_pr.iloc[:len(data)-5]
test_pr = data_pr.iloc[len(data)-5:]

Let's test algorithm with default hyperparameters. 

In [10]:
m = Prophet()
m.fit(train_pr)
future = m.make_future_dataframe(periods=5, freq = 'Y')
prophet_pred = m.predict(future)
prophet_pred = pd.DataFrame({'Date': prophet_pred[-5:]['ds'], 'Pred': prophet_pred[-5:]['yhat']})
test_pr['Prophet_Predictions'] = prophet_pred['Pred'].values
test_pr = test_pr.rename(columns={'ds': 'Date', 'y': 'Consumption'}).set_index('Date')
test_pr

INFO:numexpr.utils:Note: NumExpr detected 56 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,Consumption,Prophet_Predictions
Date,,
2015-12-31,146.6,153.806105
2016-12-31,146.9,154.704198
2017-12-31,144.3,153.967341
2018-12-31,145.2,153.536742
2019-12-31,146.0,153.413048


Predictions values are significantly higher than real consumption. What are [the parameters](https://medium.com/analytics-vidhya/time-series-analysis-using-prophet-in-python-part-1-math-explained-5936509c175c) we can tune in Prophet? Growth, changepoint prior scale, changepoint range, seasonality prior scale, holidays prior scale, seasonality mode, yearly seasonality, yearly seasonality and so on. From our point of view, increasing 'changepoint prior scale' value, that means more changepoints and more flexible trend, would be the most appropriate option. The default value is 0.05.

In [13]:
m = Prophet(changepoint_prior_scale = 0.15)
m.fit(train_pr)
future = m.make_future_dataframe(periods=5, freq = 'Y')
prophet_pred = m.predict(future)
prophet_pred = pd.DataFrame({'Date': prophet_pred[-5:]['ds'], 'Pred': prophet_pred[-5:]['yhat']})
test_pr['Prophet_Predictions'] = prophet_pred['Pred'].values
test_pr = test_pr.rename(columns={'ds': 'Date', 'y': 'Consumption'}).set_index('Date')
test_pr

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,Consumption,Prophet_Predictions
Date,,
2015-12-31,146.6,147.565860
2016-12-31,146.9,147.790017
2017-12-31,144.3,146.583190
2018-12-31,145.2,145.521119
2019-12-31,146.0,144.604480


Forecast has improved considerably, so our assumption was correct.

<h3>Evaluation</h3>

So, we take a look at our final table and visualize it. At first glance both models have good performance, but it is necessary to verify it using formal estimators. We can see a small interesting nuance in the graph below: Prophet clearly defines trend for 2015-2016 years.

In [14]:
test_pr = test_pr.drop('Consumption', axis = 1)
result = pd.concat([test, test_pr], axis = 1)
result

,Consumption,ARIMA_Predictions,Prophet_Predictions
Date,,,
2015-12-31,146.6,146.705131,147.565860
2016-12-31,146.9,146.410262,147.790017
2017-12-31,144.3,146.115393,146.583190
2018-12-31,145.2,145.820524,145.521119
2019-12-31,146.0,145.525655,144.604480


In [15]:
result.iplot(dimensions = (800,500), xTitle = 'Year', yTitle = 'Litres per capita', title = 'Beer consumption in Czech Republic: real vs predicted', color = ['#008000', '#FF9933', '#BF5B17'])

To evaluate the accuracy of the models, the next [metrics](https://medium.com/analytics-vidhya/mae-mse-rmse-coefficient-of-determination-adjusted-r-squared-which-metric-is-better-cd0326a5697e/) will be used:

- Mean Absolute Error (MAE) represents the average of the absolute difference between the actual and predicted values in the dataset. It measures the average of the residuals in the dataset.
- Mean Squared Error (MSE) represents the average of the squared difference between the original and predicted values in the data set. It measures the variance of the residuals.
- Root Mean Squared Error (RMSE) is the square root of Mean Squared Error. It measures the standard deviation of residuals.


In [16]:
arima_mae = mean_absolute_error(result['Consumption'], result['ARIMA_Predictions'])
prophet_mae = mean_absolute_error(result['Consumption'], result['Prophet_Predictions'])

arima_mse = mean_squared_error(result['Consumption'], result['ARIMA_Predictions'])
prophet_mse = mean_squared_error(result['Consumption'], result['Prophet_Predictions'])

arima_rmse = rmse(result['Consumption'], result['ARIMA_Predictions'])
prophet_rmse = rmse(result['Consumption'], result['Prophet_Predictions'])

mae = [arima_mae, prophet_mae]
mse = [arima_mse, prophet_mse]
rmse = [arima_rmse, prophet_rmse]

errors = pd.DataFrame({'Model': ['ARIMA', 'Prophet'], 'MAE': mae, 'MSE': mse, 'RMSE': rmse})
errors

,Models,MAE,MSE,RMSE
0,ARIMA,0.701026,0.831320,0.911768
1,Prophet,1.171141,1.797714,1.340788


Thus, both models demonstrated high accuracy of predictions, however, classical model could provide notably lower variance of the residuals (0.831 against 1.798).

<h2>Summary</h2>

The aim of the project was to predict annual beer consumption in the Czech Republic based on the data of the preceding years. Two models were implemented: classical time-series model - ARIMA and algorithm developed by Facebook’s Core Data Science team - Prophet. Training set contained data since 1948 and until 2014, the figures for 2015-2019 years were used as test set. To evaluate the accuracy of the models we used MAE, MSE and RMSE. Both models demonstrated high accuracy of predictions, however, classical model could provide notably lower variance of the residuals.  Though we should bear in mind small size of the initial dataset, so with an increase in years, the models may need to be adjusted.